In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/964505/CT/ct_research/v2'

In [2]:
import yaml
import numpy as np

import src.experiments.file_io as file_io
import src.experiments.aggregate_average as core
import src.experiments.shared as shared
import src.viz.aggregate_average as viz

from pathlib import Path
from datetime import datetime

from src.utils.reproducibility import set_global_seed

In [3]:
set_global_seed(42) # For reproducibility

In [4]:
data_source = "outputs/training_runs/run_20250826_percentile_v2/results.npz"
model_path = "outputs/training_runs/run_20250826_percentile_v2/model.pt"

In [5]:
run_type = "repeat"

In [6]:
test_x, test_y = file_io.load_test_data(
        file_path=data_source,
        file_names=["test_x", "test_y"]
    )
# filter for sessions with tasks that are only in one domain
test_x, sum_mask = core.filter_rows_by_sum(test_x, slice(0, 14), 1)

test_predictions = file_io.load_saved_test_predictions(
    file_path=data_source
)

test_y = test_y[sum_mask]
test_predictions = test_predictions[sum_mask]

## filter by session type
repeat_mask = core.assign_repeat(test_x)

# if run type is repeat, filter for only repeat sessions
# if run type is non-repeat, filter for only non-repeat sessions
if run_type == "non-repeat":
    repeat_mask = ~repeat_mask

test_x = test_x[repeat_mask]
test_y = test_y[repeat_mask]
test_predictions = test_predictions[repeat_mask]

In [7]:
test_x.shape, test_y.shape, test_predictions.shape

((65744, 42), (65744, 14), (65744, 14))

In [8]:
test_predictions.min()

np.float32(-0.07000223)

Model outputs negative values.  
Next steps:
- why would the model output negative values if none of the training samples contain any?
- adding an ReLU or Sigmoid activation in the last layer of the model?

- are there negative values in original?
- how small is 7%?
- clipped sigmoid in last layer?
- ground truth distribution (domain 10 in particular, all of them look different from before)
- individual cases where there are negative values

point 1: are there negative values in the original?

In [9]:
original_data_path = "outputs/training_runs/run_20250623_approximate_test_v3/results.npz"

In [10]:
original_test_predictions = file_io.load_saved_test_predictions(
    file_path=original_data_path
)

In [11]:
original_test_predictions.min()

np.float32(-0.095152125)

In [12]:
original_test_x, original_test_y = file_io.load_test_data(
        file_path=original_data_path,
        file_names=["test_x", "test_y"]
    )

In [13]:
original_test_x.min(), original_test_y.min(), original_test_predictions.min()

(np.float32(0.0), np.float32(0.0), np.float32(-0.095152125))

What about training data? Does it have negative values?

In [14]:
training_data_path = "data/encoded/predictor_encoded_20250613.npy"

In [15]:
from src.training.file_io import load_data

In [16]:
training_data = load_data(training_data_path)

Data loaded from data/encoded/predictor_encoded_20250613.npy with shape (5421080, 56)


In [ ]:
training_data.min(), training_data.max()

(np.float32(0.0), np.float32(1.0), np.float32(0.28665215))